In [3]:
! pip install --quiet pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [5]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-upswell-bug', override=True)
db_conf = get_db_conf(env)

In [ ]:
from adopt.malaria import update_ads_for_campaign


update_ads_for_campaign('28f43eeb-4fd3-4351-b46e-a8ca3f517c55', env)

In [166]:
message = {
    "message" : {
        "text": "hello",
        "quick_replies": [{"content_type": "text", "title": "Start!", "payload": "foo"}]
    }
}

message = { "message": { "text": "hello"}}

creative = {"actor_id": "102998371752603", 
            "name":"vlab-mnm-hindi-hand-black-white",
            "object_story_spec": { 
                "link_data": {
                    "call_to_action": {
                        "type": "MESSAGE_PAGE",
                        "value": {
                            "app_destination": "MESSENGER"
                        }
                    },
                    "image_hash": "ae8d36a513a20aa446701312ff9ec278",
                    "message": "Hello!",
                    "name": "foo",
                    "page_welcome_message": json.dumps(message)
                },
                "page_id":"102998371752603"
            },
            "url_tags":"ref=foo"}



In [167]:
p = {'name': 'vlab-mnm-hindi-hand-black-white', 
     'status': 'ACTIVE', 
     'creative': json.dumps(creative),
     'adset_id': '23846317632290518'}

In [160]:
p = {'name': 'vlab-mnm-hindi-hand-black-white', 
     'status': 'ACTIVE', 
     'creative': '{"actor_id":"102998371752603", "name":"vlab-mnm-hindi-hand-black-white","object_story_spec":{"link_data":{"call_to_action":{"type":"MESSAGE_PAGE","value":{"app_destination":"MESSENGER"}},"image_hash":"ae8d36a513a20aa446701312ff9ec278","message":"Hello!","name":"foo","page_welcome_message":"{\\"message\\": {\\"attachment\\": {\\"payload\\": {\\"buttons\\": [{\\"payload\\": \\"foo\\", \\"title\\": \\"Start!\\", \\"type\\": \\"postback\\"}], \\"template_type\\": \\"button\\", \\"text\\": \\"test!\\"}, \\"type\\": \\"template\\"}}}"},"page_id":"102998371752603"},"url_tags":"ref=foo"}', 
     'adset_id': '23846317632290518'}

In [ ]:

import json

print(json.dumps(p))

In [191]:
userinfo, config, db_conf, state, m, confs = load_basics('28f43eeb-4fd3-4351-b46e-a8ca3f517c55', env)

In [ ]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

# CREATING CONFS - MNM

In [ ]:
# make creatives conf
from adopt.marketing import CreativeConf


In [114]:
USER = "nandanmarkrao@gmail.com"
CAMPAIGN = "mnm-december"
CAMPAIGNID = get_campaigns_for_user(USER, db_conf)[0]["id"]

# create_campaign_for_user(USER, "foo", db_conf)

In [115]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 800000.0,
     'min_budget': 10000.0,
     'opt_window': 72,
     'end_date': '2020-01-31',
     'page_id': '102998371752603',
     'instagram_id': None,
     'ad_account': '2240415876083895',
     'ad_campaign': 'impact-evaluation-round-B-test'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [{"entity_name": None, "conf": json.dumps(config._asdict())}], db_conf)

In [113]:
import json

audience_confs = load_typed_json("config/audiences.json", List[AudienceConf])
confs = [{"entity_name": a.name, "conf": json.dumps(a._asdict())} for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [172]:
from adopt.marketing import CreativeConf

def _creative_conf(name, image, body, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "welcome_message": body,
        "link_text": "Messenger main survey shuroo karen!",
        "button_text": "Start!",
        "form": form,
    }

image_confs = [
    ('vlab-mnm-mother-daughter-voice-be-heard', '1.png', 'अपने विचार दुनिया को बताएं!'),
    ('vlab-mnm-family-flowers-keep-healthy', '2.png', 'हमें अपने स्वस्थ जीवन के राज बताएं और फ्री डाटा क्रेडिट जीतें।'),
    ('vlab-mnm-schoolgirl-laptop-got-10-min', '3.png', 'क्या आपके पास सिर्फ १० मिनिट हैं? मोबाइल डाटा क्रेडिट में २०० रुपये पाएँ। आपकी राय हमारे लिए महत्वपूर्ण है!'),
    ('vlab-mnm-grandfather-bike-healthy', '4.png', 'अपने कम्यूनिटी को स्वस्थ रखने में हमें मदद करें- हमें बताएं आप क्या सोचते है!'),
    ('vlab-mnm-woman-sari-texting-got-time', '5.png', 'एक छोटे से सर्वे के लिए आपके पास टाइम है?'),
]

creatives = [CreativeConf(**_creative_conf(name, images[n], body, 'extrabasehin')) for name, n, body in image_confs]
# creatives = load_typed_json("config/creatives.json", List[CreativeConf])
confs = [{"entity_name": c.name, "conf": json.dumps(c._asdict())} for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [ ]:
[n for n, _, _ in image_confs]

In [176]:
import pandas as pd
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

from adopt.marketing import make_stratum_conf

def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    id_ = df.disthash.iloc[0]

    return { 'id': id_,
             'metadata': {'stratumid': id_},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-mnm-mother-daughter-voice-be-heard',
                           'vlab-mnm-family-flowers-keep-healthy',
                           'vlab-mnm-schoolgirl-laptop-got-10-min',
                           'vlab-mnm-grandfather-bike-healthy',
                           'vlab-mnm-woman-sari-texting-got-time'],
             'audiences': ["vlab-mnm-kutcha-lookalike"],
             'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-cross-sectional"],
             'quota': quota, 
             'shortcodes': ['nonexisting-basleine'],
             'target_questions': [
                 {'ref': 'md:stratumid', 'op': 'equal', 'field': 'response', 'value': id_},
                 {
                     'name': 'occupation',
                     'ref': '4fc929c7-132d-49b1-a164-515e5cc9064f',
                     'op': 'answered',
                     'field': 'response',
                     'value': None
                 },
             ]}

In [177]:
df = load_cities('output/cities.csv')
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]
strata = df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 100, df)).tolist()

strata_data = [{"entity_name": c["id"], "conf": json.dumps(c)} for c in strata]

create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [26]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID)

In [31]:
from adopt.malaria import hydrate_strata, hydrate_audiences

In [ ]:
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)